In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Add this import
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# 数据集的原始路径
dataset_path = 'D:/tester01'

# 创建训练和验证文件夹
train_path = 'D:/tester01/train'
val_path = 'D:/tester01/val'
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

# 分类别进行划分
categories = ['cataract', 'normal', 'surgery']

for category in categories:
    # 创建类别的训练和验证目录
    os.makedirs(os.path.join(train_path, category), exist_ok=True)
    os.makedirs(os.path.join(val_path, category), exist_ok=True)

    # 获取所有图片路径
    images = os.listdir(os.path.join(dataset_path, category))
    images = [os.path.join(dataset_path, category, image) for image in images]

    # 划分数据集
    train_images, val_images = train_test_split(images, test_size=0.2) # 这里以80%训练，20%验证的比例划分

    # 移动图片到对应的训练和验证目录
    for image in train_images:
        shutil.move(image, os.path.join(train_path, category))

    for image in val_images:
        shutil.move(image, os.path.join(val_path, category))

# 创建一个简单的CNN模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # 假设有三个类别
])

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 数据增强
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# 准备数据集
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=64,
    class_mode='categorical')

# 训练模型
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

Found 2450 images belonging to 3 classes.
Found 614 images belonging to 3 classes.
Epoch 1/5
38/38 [==============================] - 86s 2s/step - loss: 0.8605 - accuracy: 0.5474 - val_loss: 0.6459 - val_accuracy: 0.5938
Epoch 2/5
38/38 [==============================] - 85s 2s/step - loss: 0.6954 - accuracy: 0.6287 - val_loss: 0.5814 - val_accuracy: 0.7708
Epoch 3/5
38/38 [==============================] - 85s 2s/step - loss: 0.5955 - accuracy: 0.7112 - val_loss: 0.5226 - val_accuracy: 0.8299
Epoch 4/5
38/38 [==============================] - 85s 2s/step - loss: 0.5475 - accuracy: 0.7422 - val_loss: 0.4846 - val_accuracy: 0.8281
Epoch 5/5
38/38 [==============================] - 86s 2s/step - loss: 0.5138 - accuracy: 0.7674 - val_loss: 0.4352 - val_accuracy: 0.8472


In [4]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.preprocessing import image

test_set_dir = 'D:/test set'

# 确保导入了numpy和PIL的Image类
test_image_paths = [os.path.join(test_set_dir, img) for img in os.listdir(test_set_dir)]
test_images = [np.array(Image.open(img_path).resize((299, 299))) / 255.0 for img_path in test_image_paths]
test_images = np.array(test_images, dtype=np.float32)

# 这里假设你的model变量已经是一个训练好的模型
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

# 创建类别索引到名称的映射
class_indices = {0: 'Cataract', 1: 'Normal', 2: 'Surgery'}

# 将预测的类别索引转换为类别名称
predicted_labels = [class_indices[cls] for cls in predicted_classes]

results_df = pd.DataFrame({
    'image_name': [os.path.basename(path) for path in test_image_paths],
    'predict': predicted_labels,
    'predict_proba_Normal': predictions[:, 1],
    'predict_proba_Cataract': predictions[:, 0],
    'predict_proba_Surgery': predictions[:, 2]
})

# 输出结果到Excel
results_df.to_excel('Yang.xlsx', index=False)

25/25 [==============================] - 4s 155ms/step


In [9]:
# 保存训练好的模型
model.save("Yang.h5")